In [ ]:
import sys
import pickle
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

# NOTE: This code can be downloaded from https://github.com/vv2246/tr-dag-cycles
sys.path.append("../../tr-dag-cycles/")
from cycle_utilities import tr

from collections import Counter, defaultdict

sys.path.append("../src/")
from utils import read_data
from encapsulation_dag import encapsulation_dag
from layer_randomization import layer_randomization

from matplotlib import gridspec
import matplotlib_defaults
%matplotlib inline

In [ ]:
def compute_dag_heights(dag):
    # transitiviely reduce the DAG
    dag_tr = tr(dag.copy())
    heights_dist = []
    heights_by_node = defaultdict(list)
    # Get all of the nodes that have no in-degree and non-zero out-degree
    root_nodes = [node for node in dag_tr.nodes() if dag_tr.out_degree(node) > 0 and dag_tr.in_degree(node) == 0]
    for source in root_nodes:
        sp_dict = nx.single_source_shortest_path_length(dag_tr, source)
        for target in sp_dict:
            if source == target:
                continue
            heights_dist.append(sp_dict[target])
            heights_by_node[source].append(sp_dict[target])
    return heights_dist, heights_by_node

Note that this notebook relies on running once with read_height_distribution = False to generate the data, which will take some time.

In [ ]:
data_dir = "../data/"
datasets = ["email-Enron", "contact-primary-school", "contact-high-school", "coauth-MAG-History", "coauth-MAG-Geology", "coauth-DBLP"]
num_samples = 10

In [ ]:
read_height_distributions = True

for dataset in datasets:
    print(dataset)
    if not read_height_distributions:
        observed_path = data_dir + dataset + "/" + dataset + "-" 
        print("Reading hyperedges.")
        hyperedges = read_data(observed_path, multiedges=False)

        print("Computing observed dag.")
        obs_dag, obs_nth, obs_he_map = encapsulation_dag(hyperedges)

        print(f"Dag edges: {obs_dag.number_of_edges()}")

        # Observed
        dag = obs_dag
        print("Computing observed dag heights.")
        observed_heights, obs_node_heights = compute_dag_heights(dag.copy())
        height_output_file = data_dir + dataset + f"/{dataset}_dag_heights.txt"
        with open(height_output_file, "w") as fout:
            fout.write(",".join(map(str,observed_heights)))
        print(f"Observed dag average height: {np.mean(observed_heights)}")

        obs_node_heights_file = data_dir + dataset + f"/{dataset}_dag_heights_by_node.pickle"
        with open(obs_node_heights_file, "wb") as fpickle:
            pickle.dump(obs_node_heights, fpickle)

        # Random
        random_heights = []
        for _ in range(num_samples):
            print("Computing layer randomization.")
            random_hyperedges = layer_randomization(hyperedges)
            #### Heights ####
            print("Computing random dag.")
            random_dag, _, _ = encapsulation_dag(random_hyperedges)
            print(f"Random dag has {random_dag.number_of_edges()} edges.")
            print("Computing random dag heights.")
            sample_heights, random_node_heights = compute_dag_heights(random_dag.copy())
            random_heights.append(sample_heights)
            print(f"Random dag average height: {np.mean(sample_heights)}")

        height_output_file = data_dir + dataset + f"/{dataset}_layer_randomization_dag_heights.txt"
        with open(height_output_file, "w") as fout:
            for sample_heights in random_heights:
                fout.write(",".join(map(str,sample_heights)) + "\n")
    else:
        # Get observed and random heights from files
        with open(data_dir + dataset + "/" + dataset + "_dag_heights.txt", 'r') as fin:
            observed_heights = np.array(list(map(int, fin.readline().split(','))))

        with open(data_dir + dataset + "/" + dataset + "_layer_randomization_dag_heights.txt", 'r') as fin:
            random_heights = []
            for line in fin:
                random_heights.append(np.array(list(map(int, line.split(',')))))

        with open(data_dir + dataset + "/" + dataset + "_dag_heights_by_node.pickle", "rb") as fpickle:
            obs_node_heights = pickle.load(fpickle)
    
    print()
    print()

In [ ]:
def read_height_data(dataset, data_dir="../data/"):
    dataset_info = dict()
    # Compute observed DAG
    observed_path = data_dir + dataset + "/" + dataset + "-" 
    hyperedges = read_data(observed_path, multiedges=False)
    obs_dag, obs_nth, obs_he_map = encapsulation_dag(hyperedges)
    dataset_info["obs_dag"] = obs_dag
    # Read heights by node dict
    with open(data_dir + dataset + "/" + dataset + "_dag_heights_by_node.pickle", "rb") as fpickle:
        obs_node_heights = pickle.load(fpickle)
    dataset_info["observed_node_heights"] = obs_node_heights
    
    # Get a random DAG
    random_hyperedges = layer_randomization(hyperedges)
    random_dag, _, _ = encapsulation_dag(random_hyperedges)
    dataset_info["random_dag"] = random_dag
    
    # Get observed and random heights from files
    with open(data_dir + dataset + "/" + dataset + "_dag_heights.txt", 'r') as fin:
        observed_heights = np.array(list(map(int, fin.readline().split(','))))
    dataset_info["observed_heights"] = observed_heights

    with open(data_dir + dataset + "/" + dataset + "_layer_randomization_dag_heights.txt", 'r') as fin:
        random_heights = []
        for line in fin:
            random_heights.append(np.array(list(map(int, line.split(',')))))
    dataset_info["random_heights"] = random_heights
    
    # Get averages of random count distributions
    random_count_dists = dict()
    for arr in random_heights:
        arr_counts = dict(Counter(arr))
        for key in arr_counts:
            if key in random_count_dists:
                random_count_dists[key].append(arr_counts[key])
            else:
                random_count_dists[key] = [arr_counts[key]]

    dataset_info["random_count_dists"] = random_count_dists
    
    random_means = dict()
    random_stds = dict()
    for key in random_count_dists:
        random_means[key] = np.mean(random_count_dists[key])
        random_stds[key] = np.std(random_count_dists[key])

    # Fill in missing values from both counters
    observed_counts = dict(Counter(observed_heights))
    for c in set(observed_counts.keys()).union(set(random_means.keys())):
        if c not in random_means:
            random_means[c] = 0
            random_stds[c] = 0

        if c not in observed_counts:
            observed_counts[c] = 0

    dataset_info["observed_counts"] = observed_counts
    dataset_info["random_means"] = random_means
    dataset_info["random_stds"] = random_stds
    _, random_node_heights = compute_dag_heights(random_dag.copy())
    dataset_info["random_node_heights"] = random_node_heights
    for normalized in [True, False]:
        for key, heights_by_node, dag in [("obs", obs_node_heights, obs_dag),
                                               ("rnd", random_node_heights, random_dag)]:

            # Compute DAG out-degree vs maximum height for all roots
            x = np.zeros(len(heights_by_node))
            y = np.zeros(len(heights_by_node))
            colors = np.zeros(len(heights_by_node))
            for idx, node in enumerate(heights_by_node):
                deg = len(list(dag.successors(node)))
                if not normalized:
                    x[idx] = deg
                    y[idx] = np.max(heights_by_node[node])
                else:
                    x[idx] = deg / (2**len(node)-2)
                    y[idx] = np.max(heights_by_node[node]) / (len(node)-1)              
                colors[idx] = deg
            # Sort by colors
            sorted_indices = np.argsort(colors)
            x = x[sorted_indices]
            y = y[sorted_indices]
            colors = colors[sorted_indices]
            if not normalized:
                dataset_info[key + "_x"] = list(x)
                dataset_info[key + "_y"] = list(y)
                dataset_info[key + "_colors"] = list(colors)
            else:
                dataset_info[key + "_x_normed"] = list(x)
                dataset_info[key + "_y_normed"] = list(y)
                dataset_info[key + "_colors_normed"] = list(colors)
    return dataset_info

In [ ]:
datasets = ["coauth-MAG-Geology", "coauth-MAG-History",  "contact-high-school", "contact-primary-school", "email-Enron", "email-Eu"]
dataset_info_dicts = dict()

for dataset in datasets:
    print(dataset)
    dataset_info_dicts[dataset] = read_height_data(dataset)

In [ ]:
fig = plt.figure(figsize=(56, 10), tight_layout=True)

title_size=16
inset_title_size=14
axis_label_size=16
tick_label_sizes=7
plt.rcParams['legend.fontsize'] = 14
plt.rcParams['ytick.labelsize'] = tick_label_sizes
plt.rcParams['xtick.labelsize'] = tick_label_sizes

gs = gridspec.GridSpec(3, len(datasets)*2)
for col, dataset in enumerate(datasets):
    # Get the data
    obs_node_heights = dataset_info_dicts[dataset]["observed_node_heights"]
    random_node_heights = dataset_info_dicts[dataset]["random_node_heights"]
    observed_counts = dataset_info_dicts[dataset]["observed_counts"]
    random_means = dataset_info_dicts[dataset]["random_means"]
    random_stds = dataset_info_dicts[dataset]["random_stds"]
    
    # Initialize row to 0 and get relevent axis
    row = 0
    ax = fig.add_subplot(gs[row, col])
    
    # Plot the histograms of DAG heights
    bar_width=0.15
    x = np.array(list(observed_counts.keys())) - (bar_width / 2)
    ax.bar(x, observed_counts.values(), width=bar_width, label="Observed")

    x = np.array(list(observed_counts.keys())) + (bar_width / 2)
    ax.bar(x, random_means.values(), yerr=random_stds.values(), width=bar_width, label="Random")

    ax.set(yscale='symlog', xlabel="Height", ylabel="Frequency", xlim=(-0.01, 9), xticks=list(range(1, 10)))
    if col == 0:
        ax.legend(frameon=False)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.set_title(dataset, {"fontsize":title_size})
    
    # Plot the scatter plots of degree vs height
    for normalized in [False, True]:
        row += 1
        gs_nested = gridspec.GridSpecFromSubplotSpec(nrows=1, ncols=2, subplot_spec=gs[row, col])
        max_ys = []
        max_xs = []
        nested_col = 0
        for name, heights_by_node in [("obs", obs_node_heights),
                                           ("rnd", random_node_heights)]:

            if name == "obs":
                ax = fig.add_subplot(gs_nested[0, nested_col])
            else:
                ax = fig.add_subplot(gs_nested[0, nested_col], sharey=ax)

            nested_col += 1
            
            suffix = ""
            if normalized:
                suffix = "_normed"

            x = dataset_info_dicts[dataset][name + "_x" + suffix]
            y = dataset_info_dicts[dataset][name + "_y" + suffix]
            colors = dataset_info_dicts[dataset][name + "_colors" + suffix]
            ax.scatter(x, y, c=colors, cmap='viridis', rasterized=True)
            
            # Axis labels
            if not normalized:
                ax.set_xlabel(r"$d_{\mathrm{dag}}$", fontsize=axis_label_size)
                if name == "obs":
                    ax.set_ylabel(r"$\max(h)$", fontsize=axis_label_size)
            elif normalized:
                ax.set_xlabel(r"$\frac{d_{\mathrm{dag}}}{2^k - 2}$", fontsize=axis_label_size)
                if name == "obs":
                    ax.set_ylabel(r"$\frac{\max(h)}{k-1}$", fontsize=axis_label_size)

            max_ys.append(int(max(y)))
            max_xs.append(int(max(x)))

            if normalized:
                ax.set_xticks([0.0, 0.25, 0.5, 0.75, 1.0], size=7)
                ax.set_yticks([0.0, 0.25, 0.5, 0.75, 1.0], size=7)
            else:
                ax.set(ylim=(-0.1, max(max_ys)+1), xlim=(-0.1, max(max_xs)+1))
                ax.set_yticks(list(range(2, max(max_ys)+1, 1)))
                ax.xaxis.set_major_locator(plt.MaxNLocator(5))
                if name == "obs":
                    ax.set_title(f"Observed\n({len(obs_node_heights)} roots)", size=inset_title_size)
                else:
                    ax.set_title(f"Layer Randomization\n({len(random_node_heights)} roots)", size=inset_title_size)

            ax.spines['top'].set_visible(False)
            ax.spines['right'].set_visible(False)

fig.subplots_adjust(wspace=0.9, hspace=0.3)
#fig.savefig("../results/plots/dag_analysis.pdf", bbox_inches="tight")